<a href="https://colab.research.google.com/github/abhinav9629/Sentiment-Classifier/blob/main/Sentiment_Analysis_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [2]:
import tensorflow.keras as keras
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [3]:
df = pd.read_csv('/content/MyDrive/MyDrive/HOTEL REVIEW DATASET/training.1600000.processed.noemoticon.csv',encoding='cp1252')
df.columns = ['sentiment','no','time','query','name','text']


In [4]:
df = df[['text','sentiment']]

In [5]:
df.head()

,text,sentiment
0,is upset that he can't update his Facebook by ...,0
1,@Kenichan I dived many times for the ball. Man...,0
2,my whole body feels itchy and like its on fire,0
3,"@nationwideclass no, it's not behaving at all....",0
4,@Kwesidei not the whole crew,0


In [6]:
df['text'] = df['text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',str(x)))
df['text'] = df['text'].apply(lambda x : str(x).lower())


In [7]:
tar = pd.get_dummies(df['sentiment']).values
print(tar)

[[1 0]
 [1 0]
 [1 0]
 ...
 [0 1]
 [0 1]
 [0 1]]


In [8]:
for idx,row in df.iterrows():
    row[0] = row[0].replace('rt',' ')


In [9]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['text'].values)
samples = tokenizer.texts_to_sequences(df['text'].values)
samples = pad_sequences(samples)


In [10]:
train_input,test_input,train_target,test_target = train_test_split(samples,tar,test_size=0.3)

In [11]:
#Hyperparameters
embed_dim = 128
lstm_out = 196

In [12]:
#Model

In [13]:
model = keras.Sequential([
  keras.layers.Embedding(max_features,embed_dim,input_length=train_input.shape[1]),
  keras.layers.SpatialDropout1D(0.4),
  keras.layers.LSTM(lstm_out,dropout=0.2,recurrent_dropout=0.2),
  keras.layers.Dense(2,activation='softmax')                          
])


In [14]:
optimiser = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimiser,loss="categorical_crossentropy",metrics=['accuracy'])

In [15]:
print(train_input.shape,train_target.shape)

(1119999, 40) (1119999, 2)


In [16]:
print(test_input.shape,test_target.shape)

(480000, 40) (480000, 2)


In [17]:
model.fit(train_input,train_target,batch_size=128,epochs=5)

Epoch 1/5
8750/8750 [==============================] - 1467s 167ms/step - loss: 0.4534 - accuracy: 0.7850
Epoch 2/5
8750/8750 [==============================] - 1440s 165ms/step - loss: 0.4292 - accuracy: 0.7996
Epoch 3/5
8750/8750 [==============================] - 1431s 164ms/step - loss: 0.4194 - accuracy: 0.8051
Epoch 4/5
8750/8750 [==============================] - 1424s 163ms/step - loss: 0.4125 - accuracy: 0.8088
Epoch 5/5
8750/8750 [==============================] - 1397s 160ms/step - loss: 0.4072 - accuracy: 0.8117


In [18]:
model.evaluate(test_input,test_target)

15000/15000 [==============================] - 182s 12ms/step - loss: 0.4091 - accuracy: 0.8121


[0.4090561270713806, 0.8120853900909424]

In [19]:
model.save('/content/MyDrive/MyDrive/model/sentiment_anal_tf.h5')